# load data

In [1]:
import numpy as np
import pandas as pd
import math
import json

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/pharmaHacks

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1coU_UapTjwPnfVvEmkKmrIYaovVuc4lv/pharmaHacks
/content/drive/.shortcut-targets-by-id/1coU_UapTjwPnfVvEmkKmrIYaovVuc4lv/pharmaHacks


## Extract NCT Number -> Drugbank ID -> SMILES

In [4]:
def extraction(filename='/content/drive/MyDrive/pharmaHacks/clinicaltrials_usgov_all.csv', out_drugbank="output_txt/dict_drugbank1.txt", out_allbid='output_txt/all_dbid1.txt', out_csv='output_txt/df_conversion1.csv'):
    words = ['in', 'with', 'and', 'small', 'and', 'for', 'of', 'placebo', 'molecule', 'agent', 'calcium', 'dose', 'expansion', 'arm']
    df_ct = pd.read_csv(filename, index_col=0)
    def find_all(a_str, sub):
        start = 0
        while True:
            start = a_str.find(sub, start)
            if start == -1: return
            yield start
            start += len(sub)

    dict_ct = {}
    for ii in range(df_ct.shape[0]):
        nct_no = df_ct.iloc[ii,0] # 'NCT Number'
        x = df_ct.iloc[ii,6] # 'Interventions'
        extracted = []
        if isinstance(x, str):
            temp1 = list(find_all(x, 'Drug: '))
            temp2 = list(find_all(x, '|'))
            for i1 in temp1:
                index1 = i1+6
                index2 = -1
                for i2 in temp2:
                    if i2 >= index1:
                        index2 = i2
                        break
                if index2 == -1:
                    index2 = len(x)
                temp_str = x[index1:index2]
                temp_str = temp_str.replace('(','').replace(')','').replace('+',' ').replace('/',' ')
                temp_str1 = temp_str.split()
                temp_str2 = []
                for s in temp_str1:
                    if not (len(s)<=2 or (s.lower() in words) or s.endswith('mg') or s.isnumeric()):
                        temp_str2.append(s)

                extracted.extend(temp_str2)
            extracted = list(set(extracted))
        dict_ct[nct_no] = extracted
    
    df_drugbank = pd.read_csv('/content/drive/MyDrive/pharmaHacks/drugbank_vocabulary.csv', index_col=0)
    df_drugbank['syno'] = [str(x).lower() for x in df_drugbank['Synonyms']]
    df_drugbank['common_name'] = [str(x).lower() for x in df_drugbank['Common name']]
    df_drugbank['synonyms'] = df_drugbank[['common_name', 'syno']].agg(' | '.join, axis=1)
    df_drugbank = df_drugbank.drop(columns=['syno', 'common_name'])

    dict_drugbank = {}
    all_dbid = set()
    for key in dict_ct.keys():
        db_ids = []
        for value in dict_ct.get(key):
            matches = np.where([(not math.isnan(test)) and test != -1.0 for test in df_drugbank['synonyms'].str.find(value.lower())])[0]
            li = list(df_drugbank.index[matches])
            db_ids.extend(li)
            all_dbid |= set(li)
        db_ids = list(set(db_ids))
        dict_drugbank[key] = db_ids

    json.dump(dict_drugbank, open(out_drugbank,'w'))
    with open(out_allbid, 'w') as f:
        for item in all_dbid:
            f.write("%s\n" % item)

    df_smiles = pd.read_csv('drugbank_smiles.csv')
    df_conversion_smiles = pd.DataFrame()
    for key in dict_drugbank.keys():
        for value in dict_drugbank.get(key):
            smiles = np.array(df_smiles['value'])[np.where(df_smiles['parent_key'] == value)[0]]
            if smiles.shape[0]>0:
                df_temp = {'NCT Number': key, 'DrugBank ID': value, 'SMILES': smiles[0]}
                df_conversion_smiles = df_conversion_smiles.append(df_temp, ignore_index = True)
    df_conversion_smiles.to_csv(out_csv)

    return df_ct, dict_ct, dict_drugbank, df_conversion_smiles

In [14]:
df_ct, dict_ct, dict_drugbank, df_conversion_smiles = extraction(filename='/content/drive/MyDrive/pharmaHacks/clinicaltrials_usgov_completed.csv', out_drugbank="output_txt/dict_drugbank.txt", out_allbid='output_txt/all_dbid.txt', out_csv='output_txt/df_conversion.csv')
df_ct2, dict_ct2, dict_drugbank2, df_conversion_smiles2 = extraction(filename='/content/drive/MyDrive/pharmaHacks/clinicaltrials_usgov_all.csv', out_drugbank="output_txt/dict_drugbank1.txt", out_allbid='output_txt/all_dbid1.txt', out_csv='output_txt/df_conversion1.csv')

In [15]:
df_conversion_smiles2

,NCT Number,DrugBank ID,SMILES
0,NCT02179671,DB11689,CN1C=NC2=C(F)C(NC3=C(Cl)C=C(Br)C=C3)=C(C=C12)C...
1,NCT02179671,DB00317,COC1=C(OCCCN2CCOCC2)C=C2C(NC3=CC(Cl)=C(F)C=C3)...
2,NCT02179671,DB01248,[H][C@@]1(C[C@@]2(O)[C@@H](OC(=O)C3=CC=CC=C3)[...
3,NCT02259114,DB15189,CC1=C(C)C2=C(S1)N1C(C)=NN=C1[C@H](CC(=O)NC1=CC...
4,NCT02395601,DB14581,COC1=C(CNC(=O)C2=C(C)N([C@H](C)C3CCN(CC(F)(F)F...
...,...,...,...
3694,NCT01145209,DB02309,O[C@H]1[C@@H](O)[C@@H](O[C@@H]1COP(O)(O)=O)[N+...
3695,NCT01145209,DB02548,OC[C@H](O)[C@@H](O)[C@H](O)[C@H](O)COP(O)(O)=O
3696,NCT01145209,DB04083,CC1=NC=C(COP(O)(O)=O)C(\C=N\CCCC[C@H](N)C(O)=O...
3697,NCT01145209,DB04143,[H][C@@](O)(COP(O)(O)=O)[C@@]([H])(O)C1=CNC2=C...


In [7]:
df_ct

,NCT Number,Title,Acronym,Status,Study Results,Conditions,Interventions,Outcome Measures,Sponsor/Collaborators,Gender,...,Other IDs,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents,URL
Rank,,,,,,,,,,,,,,,,,,,,,
1,NCT00579683,Registry for Patients With Acquired Resistance...,NaN,Completed,No Results Available,Unresectable or Metastatic Non-small Cell Lung...,Other: Tumor core biopsy for RNA isolation,To compare EGFR gene sequence in patients upon...,Memorial Sloan Kettering Cancer Center,All,...,04-103,August 2004,February 2016,February 2016,"December 24, 2007",NaN,"February 4, 2016","Memoral Sloan Kettering Cancer Center, Basking...",NaN,https://ClinicalTrials.gov/show/NCT00579683
2,NCT00671671,Phase 1 Study To Evaluate Antiviral Activity O...,NaN,Completed,Has Results,"Hepatitis, Chronic|Hepatitis C Virus",Drug: Small Molecule Agent (PF-868554),Change From Baseline in Plasma Log10 Hepatitis...,Pfizer,All,...,A8121006,April 2008,December 2008,December 2008,"May 5, 2008","January 14, 2014","January 14, 2014","Pfizer Investigational Site, Gainesville, Flor...",NaN,https://ClinicalTrials.gov/show/NCT00671671
3,NCT02179671,Immune-Modulated Study of Selected Small Molec...,NaN,Completed,Has Results,Locally Advanced or Metastatic Non-Small-Cell ...,Drug: Gefitinib|Drug: AZD9291|Drug: Selumetini...,Confirmed Complete Response (CR) Rate|Objectiv...,"AstraZeneca|Quintiles, Inc.",All,...,D4191C00011,"July 25, 2014","June 11, 2016","June 11, 2016","July 2, 2014","August 2, 2019","August 2, 2019","Research Site, Goodyear, Arizona, United State...",NaN,https://ClinicalTrials.gov/show/NCT02179671
4,NCT02259114,"A Dose-Finding Study of Birabresib (MK-8628), ...",NaN,Completed,Has Results,NUT Midline Carcinoma|Triple Negative Breast C...,Drug: Birabresib,Number of Participants Who Experienced a Dose ...,Oncoethix GmbH,All,...,8628-003|OTX015_108|2014-002680-15|MK-8628-003,"October 23, 2014","March 3, 2017","March 3, 2017","October 8, 2014","October 1, 2018","January 26, 2021",NaN,"""Study Protocol"", https://ClinicalTrials.gov/P...",https://ClinicalTrials.gov/show/NCT02259114
5,NCT02395601,A Study Evaluating CPI-1205 in Patients With B...,NaN,Completed,No Results Available,B-Cell Lymphoma,Drug: CPI-1205,Frequency of Dose-limiting toxicities (DLTs)|F...,Constellation Pharmaceuticals,All,...,1205-01,March 2015,December 2018,December 2018,"March 23, 2015",NaN,"September 6, 2019","Horizon Oncology Center, Lafayette, Indiana, U...",NaN,https://ClinicalTrials.gov/show/NCT02395601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,NCT01306149,Natural History of Mineral Metabolism Paramete...,NaN,Completed,No Results Available,Complication of Peritoneal Dialysis,NaN,natural history of clearances and serum parame...,Universitaire Ziekenhuizen Leuven,All,...,FWO01-03-21,July 2002,"January 31, 2017","January 31, 2017","March 1, 2011",NaN,"October 25, 2017","University Hospital, Leuven, Vlaams-Brabant, B...",NaN,https://ClinicalTrials.gov/show/NCT01306149
284,NCT00935792,Everolimus and Alemtuzumab in Treating Patient...,NaN,Completed,Has Results,Lymphocytic Leukemia,Drug: alemtuzumab|Drug: everolimus,Clinical Response (Complete or Partial Remissi...,Mayo Clinic,All,...,MC088C|NCI-2009-00935|08-008775,July 2009,December 2015,December 2015,"July 9, 2009","December 12, 2016","September 26, 2017","Mayo Clinic in Arizona, Scottsdale, Arizona, U...",NaN,https://ClinicalTrials.gov/show/NCT00935792
285,NCT01456767,Effects of Administration of L.Plantarum WCFS1...,NaN,Completed,No Results Available,Gastro Intestinal Infection,Dietary Supplement: Probiotics,"Change in sugar recovery in urine, as indicato...",Maastricht University Medical Center,All,...,NL35728.068.11/METC11-2-011,October 2011,April 2012,NaN,"October 21, 2011",NaN,"July 19, 2012","Health, Medicine and Life Sciences, Maastricht...",NaN,https://ClinicalTrials.gov/show/NCT01456767


In [9]:
df_ct2

,NCT Number,Title,Acronym,Status,Study Results,Conditions,Interventions,Outcome Measures,Sponsor/Collaborators,Gender,...,Other IDs,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents,URL
Rank,,,,,,,,,,,,,,,,,,,,,
1,NCT02460939,The Study on the Pulmonary Vessel Disease and ...,NaN,Unknown status,No Results Available,Pulmonary Vessel Disease,NaN,The changes of patients'metabolism of small mo...,Beijing Chao Yang Hospital,All,...,12704102,August 2014,January 2016,August 2016,"June 3, 2015",NaN,"June 3, 2015","Liu Sijie, Beijing, Beijing, China",NaN,https://ClinicalTrials.gov/show/NCT02460939
2,NCT00579683,Registry for Patients With Acquired Resistance...,NaN,Completed,No Results Available,Unresectable or Metastatic Non-small Cell Lung...,Other: Tumor core biopsy for RNA isolation,To compare EGFR gene sequence in patients upon...,Memorial Sloan Kettering Cancer Center,All,...,04-103,August 2004,February 2016,February 2016,"December 24, 2007",NaN,"February 4, 2016","Memoral Sloan Kettering Cancer Center, Basking...",NaN,https://ClinicalTrials.gov/show/NCT00579683
3,NCT03812055,Cellular Pharmacodynamics of Small Molecules i...,NaN,Unknown status,No Results Available,Lysosomal Storage Diseases,NaN,Effect on enzyme activity|Effect on substrate ...,Lysosomal and Rare Disorders Research and Trea...,All,...,18-LDRTC-02,"July 6, 2018",July 2020,July 2020,"January 22, 2019",NaN,"January 22, 2019","LDRTC, Fairfax, Virginia, United States",NaN,https://ClinicalTrials.gov/show/NCT03812055
4,NCT00671671,Phase 1 Study To Evaluate Antiviral Activity O...,NaN,Completed,Has Results,"Hepatitis, Chronic|Hepatitis C Virus",Drug: Small Molecule Agent (PF-868554),Change From Baseline in Plasma Log10 Hepatitis...,Pfizer,All,...,A8121006,April 2008,December 2008,December 2008,"May 5, 2008","January 14, 2014","January 14, 2014","Pfizer Investigational Site, Gainesville, Flor...",NaN,https://ClinicalTrials.gov/show/NCT00671671
5,NCT05282953,A Phase I/II Dose-escalating Study of the Safe...,ABACUS,Not yet recruiting,No Results Available,Retinitis Pigmentosa,Drug: KIO-301,Number of participants with treatment-related ...,"Kiora Pharmaceuticals, Inc.",All,...,KIO-301-1101,August 2022,February 2023,March 2023,"March 16, 2022",NaN,"March 16, 2022",NaN,NaN,https://ClinicalTrials.gov/show/NCT05282953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,NCT02393274,Microcirculation in Venoarterial Extracorporea...,NaN,Completed,No Results Available,Heart Failure|Respiratory Failure,Device: Extra-corporeal membrane oxygenation l...,Perfused small vessel density|Total small vess...,National Taiwan University Hospital,All,...,201412045RINA,April 2015,August 2016,December 2016,"March 19, 2015",NaN,"April 23, 2018","National Taiwan University Hospital, Taipei, T...",NaN,https://ClinicalTrials.gov/show/NCT02393274
808,NCT02872961,"SIBO, Immune Activation, and FGIDs in Children",SIBO,Completed,No Results Available,Small Intestinal Bacterial Overgrowth|Irritabl...,NaN,Abdominal Pain Index|Duodenal fluid microbiota...,"The University of Texas Health Science Center,...",All,...,HSC-MS-11-0726,April 2013,"February 13, 2017","February 13, 2017","August 19, 2016",NaN,"March 29, 2021",The University of Texas of Health Science Cent...,NaN,https://ClinicalTrials.gov/show/NCT02872961
809,NCT04513002,Ataxia-telangiectasia: Treating Mitochondrial ...,A-TC7,Not yet recruiting,No Results Available,Ataxia Telangiectasia,Dietary Supplement: Triheptanoin,Nasal epithelial cell survival under condition...,The University of Queensland|National Health a...,All,...,A-T2020/01,February 2022,June 2023,May 2024,"August 14, 2020",NaN,"February 23, 2022","Queensland Children's Hospital, Brisbane, Quee...",NaN,https://ClinicalTrials.gov/show/NCT04513002


In [13]:
df_ct_completed = pd.merge(df_conversion_smiles, df_ct)
df_ct_completed = df_ct_completed.drop(columns=['Acronym'])
df_ct_completed.to_csv('output_txt/df_ct_completed.csv')
df_ct_completed

,NCT Number,DrugBank ID,SMILES,Title,Status,Study Results,Conditions,Interventions,Outcome Measures,Sponsor/Collaborators,...,Other IDs,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents,URL
0,NCT02179671,DB11689,CN1C=NC2=C(F)C(NC3=C(Cl)C=C(Br)C=C3)=C(C=C12)C...,Immune-Modulated Study of Selected Small Molec...,Completed,Has Results,Locally Advanced or Metastatic Non-Small-Cell ...,Drug: Gefitinib|Drug: AZD9291|Drug: Selumetini...,Confirmed Complete Response (CR) Rate|Objectiv...,"AstraZeneca|Quintiles, Inc.",...,D4191C00011,"July 25, 2014","June 11, 2016","June 11, 2016","July 2, 2014","August 2, 2019","August 2, 2019","Research Site, Goodyear, Arizona, United State...",NaN,https://ClinicalTrials.gov/show/NCT02179671
1,NCT02179671,DB00317,COC1=C(OCCCN2CCOCC2)C=C2C(NC3=CC(Cl)=C(F)C=C3)...,Immune-Modulated Study of Selected Small Molec...,Completed,Has Results,Locally Advanced or Metastatic Non-Small-Cell ...,Drug: Gefitinib|Drug: AZD9291|Drug: Selumetini...,Confirmed Complete Response (CR) Rate|Objectiv...,"AstraZeneca|Quintiles, Inc.",...,D4191C00011,"July 25, 2014","June 11, 2016","June 11, 2016","July 2, 2014","August 2, 2019","August 2, 2019","Research Site, Goodyear, Arizona, United State...",NaN,https://ClinicalTrials.gov/show/NCT02179671
2,NCT02179671,DB01248,[H][C@@]1(C[C@@]2(O)[C@@H](OC(=O)C3=CC=CC=C3)[...,Immune-Modulated Study of Selected Small Molec...,Completed,Has Results,Locally Advanced or Metastatic Non-Small-Cell ...,Drug: Gefitinib|Drug: AZD9291|Drug: Selumetini...,Confirmed Complete Response (CR) Rate|Objectiv...,"AstraZeneca|Quintiles, Inc.",...,D4191C00011,"July 25, 2014","June 11, 2016","June 11, 2016","July 2, 2014","August 2, 2019","August 2, 2019","Research Site, Goodyear, Arizona, United State...",NaN,https://ClinicalTrials.gov/show/NCT02179671
3,NCT02259114,DB15189,CC1=C(C)C2=C(S1)N1C(C)=NN=C1[C@H](CC(=O)NC1=CC...,"A Dose-Finding Study of Birabresib (MK-8628), ...",Completed,Has Results,NUT Midline Carcinoma|Triple Negative Breast C...,Drug: Birabresib,Number of Participants Who Experienced a Dose ...,Oncoethix GmbH,...,8628-003|OTX015_108|2014-002680-15|MK-8628-003,"October 23, 2014","March 3, 2017","March 3, 2017","October 8, 2014","October 1, 2018","January 26, 2021",NaN,"""Study Protocol"", https://ClinicalTrials.gov/P...",https://ClinicalTrials.gov/show/NCT02259114
4,NCT02395601,DB14581,COC1=C(CNC(=O)C2=C(C)N([C@H](C)C3CCN(CC(F)(F)F...,A Study Evaluating CPI-1205 in Patients With B...,Completed,No Results Available,B-Cell Lymphoma,Drug: CPI-1205,Frequency of Dose-limiting toxicities (DLTs)|F...,Constellation Pharmaceuticals,...,1205-01,March 2015,December 2018,December 2018,"March 23, 2015",NaN,"September 6, 2019","Horizon Oncology Center, Lafayette, Indiana, U...",NaN,https://ClinicalTrials.gov/show/NCT02395601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,NCT01453153,DB12564,CCCCCCCC\C=C\CCCCCCCC(=O)OC[C@H]1O[C@@H](N2C=C...,Study of Gemcitabine + PEGPH20 vs Gemcitabine ...,Completed,Has Results,Stage IV Pancreatic Cancer,Drug: Gemcitabine|Drug: PEGPH20|Drug: Placebo,Number of Participants With a Dose-limiting To...,Halozyme Therapeutics,...,Halo-109-201,September 2011,February 2014,May 2015,"October 17, 2011","November 30, 2018","November 30, 2018","Highlands Oncology Group, Fayetteville, Arkans...",NaN,https://ClinicalTrials.gov/show/NCT01453153
917,NCT01729728,DB06204,CC[C@H]([C@@H](C)CN(C)C)C1=CC(O)=CC=C1,"Pharmacokinetic, Efficacy and Safety Study of ...",Completed,Has Results,Postoperative Pain|Acute Pain,Drug: Tapentadol,Pharmacokinetic Profile of Serum Concentration...,Grünenthal GmbH|Janssen Research & Development...,...,KF5503/68|2013-002016-27,November 2012,February 2014,February 2014,"November 20, 2012","November 27, 2014","November 4, 2019","Jean Brown Research, Salt Lake City, Utah, Uni...",NaN,https://ClinicalTrials.gov/show/NCT01729728
918,NCT01638546,DB0

In [16]:
df_ct_completed2 = pd.merge(df_conversion_smiles2, df_ct2)
df_ct_completed2 = df_ct_completed2.drop(columns=['Acronym'])
df_ct_completed2.to_csv('output_txt/df_ct_all.csv')
df_ct_completed2

,NCT Number,DrugBank ID,SMILES,Title,Status,Study Results,Conditions,Interventions,Outcome Measures,Sponsor/Collaborators,...,Other IDs,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents,URL
0,NCT02179671,DB11689,CN1C=NC2=C(F)C(NC3=C(Cl)C=C(Br)C=C3)=C(C=C12)C...,Immune-Modulated Study of Selected Small Molec...,Completed,Has Results,Locally Advanced or Metastatic Non-Small-Cell ...,Drug: Gefitinib|Drug: AZD9291|Drug: Selumetini...,Confirmed Complete Response (CR) Rate|Objectiv...,"AstraZeneca|Quintiles, Inc.",...,D4191C00011,"July 25, 2014","June 11, 2016","June 11, 2016","July 2, 2014","August 2, 2019","August 2, 2019","Research Site, Goodyear, Arizona, United State...",NaN,https://ClinicalTrials.gov/show/NCT02179671
1,NCT02179671,DB00317,COC1=C(OCCCN2CCOCC2)C=C2C(NC3=CC(Cl)=C(F)C=C3)...,Immune-Modulated Study of Selected Small Molec...,Completed,Has Results,Locally Advanced or Metastatic Non-Small-Cell ...,Drug: Gefitinib|Drug: AZD9291|Drug: Selumetini...,Confirmed Complete Response (CR) Rate|Objectiv...,"AstraZeneca|Quintiles, Inc.",...,D4191C00011,"July 25, 2014","June 11, 2016","June 11, 2016","July 2, 2014","August 2, 2019","August 2, 2019","Research Site, Goodyear, Arizona, United State...",NaN,https://ClinicalTrials.gov/show/NCT02179671
2,NCT02179671,DB01248,[H][C@@]1(C[C@@]2(O)[C@@H](OC(=O)C3=CC=CC=C3)[...,Immune-Modulated Study of Selected Small Molec...,Completed,Has Results,Locally Advanced or Metastatic Non-Small-Cell ...,Drug: Gefitinib|Drug: AZD9291|Drug: Selumetini...,Confirmed Complete Response (CR) Rate|Objectiv...,"AstraZeneca|Quintiles, Inc.",...,D4191C00011,"July 25, 2014","June 11, 2016","June 11, 2016","July 2, 2014","August 2, 2019","August 2, 2019","Research Site, Goodyear, Arizona, United State...",NaN,https://ClinicalTrials.gov/show/NCT02179671
3,NCT02259114,DB15189,CC1=C(C)C2=C(S1)N1C(C)=NN=C1[C@H](CC(=O)NC1=CC...,"A Dose-Finding Study of Birabresib (MK-8628), ...",Completed,Has Results,NUT Midline Carcinoma|Triple Negative Breast C...,Drug: Birabresib,Number of Participants Who Experienced a Dose ...,Oncoethix GmbH,...,8628-003|OTX015_108|2014-002680-15|MK-8628-003,"October 23, 2014","March 3, 2017","March 3, 2017","October 8, 2014","October 1, 2018","January 26, 2021",NaN,"""Study Protocol"", https://ClinicalTrials.gov/P...",https://ClinicalTrials.gov/show/NCT02259114
4,NCT02395601,DB14581,COC1=C(CNC(=O)C2=C(C)N([C@H](C)C3CCN(CC(F)(F)F...,A Study Evaluating CPI-1205 in Patients With B...,Completed,No Results Available,B-Cell Lymphoma,Drug: CPI-1205,Frequency of Dose-limiting toxicities (DLTs)|F...,Constellation Pharmaceuticals,...,1205-01,March 2015,December 2018,December 2018,"March 23, 2015",NaN,"September 6, 2019","Horizon Oncology Center, Lafayette, Indiana, U...",NaN,https://ClinicalTrials.gov/show/NCT02395601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3694,NCT01145209,DB02309,O[C@H]1[C@@H](O)[C@@H](O[C@@H]1COP(O)(O)=O)[N+...,Ofatumumab-based Induction Chemoimmunotherapy ...,"Active, not recruiting",Has Results,Small Lymphocytic Lymphoma|CLL (Chronic Lympho...,Drug: Fludarabine Phosphate|Biological: Ofatum...,Progression Free Survival Rate 2 Years After I...,"National Heart, Lung, and Blood Institute (NHL...",...,100141|10-H-0141,"July 1, 2010","October 31, 2017","December 31, 2025","June 16, 2010","September 17, 2019","March 8, 2021","National Institutes of Health Clinical Center,...","""Study Protocol and Statistical Analysis Plan""...",https://ClinicalTrials.gov/show/NCT01145209
3695,NCT01145209,DB02548,OC[C@H](O)[C@@H](O)[C@H](O)[C@H](O)COP(O)(O)=O,Ofatumumab-based Induction Chemoimmunotherapy ...,"Active, not recruiting",Has Results,Small Lymphocytic Lymphoma|CLL (Chronic Lympho...,Drug: Fludarabine Phosphate|Biological: Ofatum...,Progression Free Survival Rate 2 Years After I...,"National Heart, Lung, and Blood Institute (NHL...",...,100141|10-H-0141,"July 1, 2010"